# 6장 Google Teachable Machine을 사용한 졸지마 프로젝트

## 6.2.4 라이브러리 설치하기

- !pip install opencv-python 
- !pip uninstall tensorflow
- !pip install tensorflow>=2.3
- !pip install Pillow
- !pip install beepy

## 6.3 사전 지식 쌓기

##  6.3.1 OpenCV로 카메라 입력 받기
- OpenCV란? https://opencv-python.readthedocs.io/en/latest/

In [ ]:
import cv2

# 카메라 캡처 객체, 0=내장 카메라
capture = cv2.VideoCapture(0)

# 캡처 프레임 사이즈 조절
capture.set(cv2.CAP_PROP_FRAME_WIDTH, 320)
capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 240)

while True: # 특정 키를 누를 때까지 무한 반복
    # 한 프레임씩 읽기
    ret, frame = capture.read()
    if ret == True: 
        print("read success!")

    # 이미지 뒤집기, 1=좌우 뒤집기
    frame_fliped = cv2.flip(frame, 1)

    # 읽어들인 프레임을 윈도우 창에 출력
    cv2.imshow("VideoFrame", frame_fliped)

    # 1ms 동안 사용자가 키를 누르기를 기다림
    if cv2.waitKey(1) > 0: 
        break 

# 카메라 객체 반환
capture.release()

# 화면에 나타난 윈도우를 종료
cv2.destroyAllWindows()

## 6.3.2 Google Teachable Machine 이해하기
- Google Teachable Machine 공식 홈페이지: https://teachablemachine.withgoogle.com

# 6.4 구현하기

## Step1) 조는 모습을 판별하는 모델 생성하기

In [ ]:
import tensorflow.keras
from PIL import Image, ImageOps
import numpy as np

# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

# Load the model
model = tensorflow.keras.models.load_model('res/dont_sleep/converted_keras/keras_model.h5')

# Create the array of the right shape to feed into the keras model
# The 'length' or number of images you can put into the array is
# determined by the first position in the shape tuple, in this case 1.
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

# Replace this with the path to your image
image = Image.open('res/dont_sleep/test_photo.jpg')

#resize the image to a 224x224 with the same strategy as in TM2:
#resizing the image to be at least 224x224 and then cropping from the center
size = (224, 224)
image = ImageOps.fit(image, size, Image.ANTIALIAS)

#turn the image into a numpy array
image_array = np.asarray(image)

# display the resized image
image.show()

# Normalize the image
normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1

# Load the image into the array
data[0] = normalized_image_array

# run the inference
prediction = model.predict(data)
print(prediction)

In [ ]:
prediction[0, 1]

## Step2) 잠깨는 방법 설정하기

In [ ]:
import beepy
import kakao_utils

# 컴퓨터에 내장된 소리를 출력
def beepsound():
    beepy.beep(sound=6)

# 카카오톡 메시지로 '졸음 방지 베타파' 영상 링크를 전송
def send_music_link():
    KAKAO_TOKEN_FILENAME = "res/kakao_message/kakao_token.json"
    KAKAO_APP_KEY = "<REST_API 앱키를 입력하세요>"
    tokens = kakao_utils.update_tokens(KAKAO_APP_KEY, KAKAO_TOKEN_FILENAME)


    # 텍스트 메시지 보내기
    template = {
        "object_type": "text",
        "text": "당신은 30초 이상 졸았습니다. 졸지 마세요!!!!",
        "link": {
            "web_url": "https://www.youtube.com/watch?v=7Q2N7919o5o",
            "mobile_web_url": "https://www.youtube.com/watch?v=7Q2N7919o5o"
        },
        "button_title": "잠깨는 노래 듣기"
    }

    # 카카오톡 메시지 전송
    res = kakao_utils.send_message(KAKAO_TOKEN_FILENAME, template)
    if res.json().get('result_code') == 0:
        print('텍스트 메시지를 성공적으로 보냈습니다.')
    else:
        print('텍스트 메시지를 보내지 못했습니다. 오류메시지 : ', res.json())

In [ ]:
#beepsound()
#send_music_link()

## Step3) 카메라 촬영 영상으로 조는 상태 감지하기

In [ ]:
import cv2
import tensorflow.keras
import numpy as np

## 이미지 전처리
def preprocessing(frame):
    # 크기 조정
    size = (224, 224)
    frame_resized = cv2.resize(frame, size, interpolation=cv2.INTER_AREA)

    # 이미지 정규화
    frame_normalized = (frame_resized.astype(np.float32) / 127.0) - 1

    # 이미지 차원 재조정 - 예측을 위해 reshape 해줍니다.
    frame_reshaped = frame_normalized.reshape((1, 224, 224, 3))

    return frame_reshaped

## 학습된 모델 불러오기
model_filename = 'res/dont_sleep/converted_keras/keras_model.h5'
model = tensorflow.keras.models.load_model(model_filename)

# 카메라 캡처 객체, 0=내장 카메라
capture = cv2.VideoCapture(0)

# 캡처 프레임 사이즈 조절
capture.set(cv2.CAP_PROP_FRAME_WIDTH, 320)
capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 240)

sleep_cnt = 1 # 30초간 "졸림" 상태를 확인하기 위한 변수
while True:
    ret, frame = capture.read()
    if ret == True:
        print("read success!")

    # 이미지 뒤집기
    frame_fliped = cv2.flip(frame, 1)

    # 이미지 출력
    cv2.imshow("VideoFrame", frame_fliped)

    # 1초마다 검사하며, videoframe 창으로 아무 키나 누르게 되면 종료
    if cv2.waitKey(200) > 0:
        break

    # 데이터 전처리
    preprocessed = preprocessing(frame_fliped)

    # 예측
    prediction = model.predict(preprocessed)
    #print(prediction) # [[0.00533728 0.99466264]]

    if prediction[0,0] < prediction[0,1]:
        print('졸림 상태')
        sleep_cnt += 1

        # 졸린 상태가 30초간 지속되면 소리 & 카카오톡 보내기
        if sleep_cnt % 30 == 0:
            sleep_cnt = 1
            print('30초간 졸고 있네요!!!')
            beepsound()
            send_music_link()
            break ## 1번만 알람이 오면 프로그램을 정지 시킴 (반복을 원한다면, 주석으로 막기!)
    else:
        print('깨어있는 상태')
        sleep_cnt = 1

# 카메라 객체 반환
capture.release()
# 화면에 나타난 윈도우 창을 종료
cv2.destroyAllWindows()